<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# World Situation Dashboard 

**Tags:** #dashboard #plotly #dash #naas #asset #automation #ai #analytics

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/)

This notebook enables you to generate a dashboard to follow the world situation.

## Input

### Import libraries

In [ ]:
import os
from naas_drivers import gsheet
import naas
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
from dash_bootstrap_components._components.Container import Container
import pandas as pd
from plotly.subplots import make_subplots
from dash.exceptions import PreventUpdate

### Defining the port of the dashboard

In [ ]:
DASH_PORT = 8050

### Setup APP

In [ ]:
APP_TITLE = 'World Situation Room'
APP_LOGO = "https://pbs.twimg.com/profile_images/1243203211114274817/WqWaa9Bm_400x400.jpg"

### Setup Google Sheets
- Share your Google Sheets spreadsheet with our service account : 🔗 naas-share@naas-gsheets.iam.gserviceaccount.com

In [ ]:
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1yi0qzuUEnE9wMWWQFIVq5Uou8ChxFqF0MMqFM5FtVIM"
SHEET_NAME = "KPIS"

## Model

### Data

#### Highlighted KPIs

In [ ]:
df_hkpis = gsheet.connect(SPREADSHEET_URL).get(sheet_name=SHEET_NAME)
df_hkpis["VALUE_D"] = df_hkpis["VALUE"].astype(str) + " " + df_hkpis["UNITS"]
df_hkpis

#### Barline 

In [ ]:
data = [
    {"ENTITY": "Worldwide", "SCENARIO": "2022", "DATE": "202201", "VALUE": 130, "VARV": 130},
    {"ENTITY": "Worldwide", "SCENARIO": "2022", "DATE": "202202", "VALUE": 180, "VARV": 50},
    {"ENTITY": "Worldwide", "SCENARIO": "2022", "DATE": "202203", "VALUE": 190, "VARV": 10},
    {"ENTITY": "Worldwide", "SCENARIO": "2022", "DATE": "202204", "VALUE": 200, "VARV": 10},
    {"ENTITY": "Worldwide", "SCENARIO": "2022", "DATE": "202205", "VALUE": 280, "VARV": 80},
    {"ENTITY": "Worldwide", "SCENARIO": "2021", "DATE": "202101", "VALUE": 80, "VARV": 80},
    {"ENTITY": "Worldwide", "SCENARIO": "2021", "DATE": "202102", "VALUE": 180, "VARV": 100},
    {"ENTITY": "Worldwide", "SCENARIO": "2021", "DATE": "202103", "VALUE": 230, "VARV": 50},
    {"ENTITY": "Worldwide", "SCENARIO": "2021", "DATE": "202104", "VALUE": 240, "VARV": 10},
    {"ENTITY": "Worldwide", "SCENARIO": "2021", "DATE": "202105", "VALUE": 320, "VARV": 80},
]
df_barline = pd.DataFrame(data)
df_barline

### Design

#### Create dropdown "Entity"

In [ ]:
entities = [
    "Worldwide",
    "France",
    "UK",
    "Spain",
    "Belgium",
    "Italy"
]

dropdown_entity = dcc.Dropdown(
    id='entity',
    options=[{'label': i, 'value': i} for i in entities],
    placeholder='Entity',
    value=entities[0],
)

#### Create dropdown "Scenario"

In [ ]:
scenarios = [
    "2022",
    "2021",
    "2020",
    "2019"
]

dropdown_scenario = dcc.Dropdown(
    id='scenario',
    options=[{'label': i, 'value': i} for i in scenarios],
    placeholder='Scenario',
    value=scenarios[0],
)

#### Create Navbar

In [ ]:
navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src=APP_LOGO, height="30px")),
                        dbc.Col(dbc.NavbarBrand(APP_TITLE, className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://mobile.twitter.com/ws_room/photo",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            dbc.Collapse(
                dbc.Nav(
                    [
                        html.Div(
                            [
                                html.Div(className="w-100"),
                                html.Div(className="w-100"),
                                html.Div(dropdown_entity, className="w-100"),
                                html.Div(dropdown_scenario, className="w-100")
                            ],
                            className="pt-1 pb-1 d-grid gap-2 d-md-flex w-100")

                    ],
                    className="ms-auto w-100",
                    navbar=True,
                ),
                id="navbar-collapse",
                navbar=True,
                is_open=False,
            ),
        ],
    ),
    color="dark",
    dark=True,
)

#### Create Cards

In [ ]:
#Style used for card
CARD_COL_STYLE = {
    "className": "gx-5 g-2",
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 2,
    "xl": 2
}

#Function to create card
def create_card_col(uid, card_title, card_paragraph):
    card = dbc.Col(
        dbc.Card(
            dbc.CardBody(
                [
#                     html.H5(card_title, className="card-title"),
                    html.P(card_title, className="card-title", style={"font-size": "18px", "padding-bottom": "5px"}),
                    html.P(card_paragraph, className="card-text", id=uid)
                ]
            ),
            color="light",
            inverse=False
        ),
        **CARD_COL_STYLE
    )
    return card

#### Create Chart

In [ ]:
#Style used for chart
CHART_COL_STYLE = {
    "xs": 12,
    "sm": 12,
    "md": 12,
    "lg": 6,
    "xl": 6
}

##### Barline chart

In [ ]:
def create_barlinechart(df,
                        label="DATE",
                        value="VALUE",
                        varv="VARV",
                        xaxis_title="Months",
                        yaxis_title_r=None,
                        yaxis_title_l=None):
    
    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add traces
    fig.add_trace(
        go.Bar(
            x=df[label],
            y=df[varv],
            marker=dict(color="#ADD8E6"),
        ),
        secondary_y=False,
    )
    fig.add_trace(
        go.Scatter(
            x=df[label],
            y=df[value],
            mode="lines",
            line=dict(color="#0A66C2", width=2.5),
        ),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title="Plotly - Barline chart",
        title_font=dict(family="Arial", size=18, color="black"),
        legend=None,
        plot_bgcolor="#ffffff",
        paper_bgcolor="white",
        xaxis_title=xaxis_title,
        xaxis_title_font=dict(family="Arial", size=10, color="black"),
    )

    # Set y-axes titles
    fig.update_yaxes(
        title_text=yaxis_title_r,
        title_font=dict(family="Arial", size=10, color="black"),
        secondary_y=False
    )
    fig.update_yaxes(
        title_text=yaxis_title_l,
        title_font=dict(family="Arial", size=10, color="black"),
        secondary_y=True
    )
    fig.update_traces(showlegend=False)
    fig.show()
    return fig


# Create dataframe to test function
df_barline_test = df_barline.copy()
df_barline_test = df_barline_test[(df_barline_test.ENTITY == entities[0]) & 
                                  (df_barline_test.SCENARIO == scenarios[0])]

# Display result
fig = create_barlinechart(df_barline_test,
                          yaxis_title_r="Variation",
                          yaxis_title_l="Value")

### Dash App

In [ ]:
# Init App
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/', 
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[{'name': 'viewport','content': 'width=device-width, initial-scale=1.0'}]
                
)

# App title
app.title = APP_TITLE

# App layout
app.layout = html.Div(
    [
        #Navbar
        navbar,
        
        #HKPIS
        html.Div(
            [
                dbc.Row(
                    [
                        create_card_col("card1", "🧬 Carbon Dioxyde", "419 ppm"),
                        create_card_col("card2", "🌡 Temperature", "+0.85°C"),
                        create_card_col("card3", "❄️ Artic Sea Ice", "-13% per decade"),
                        create_card_col("card4", "🧊 Ice Sheet", "-152 bilions metric"),
                        create_card_col("card5", "🌊 Sea Level", "+1O1 mm"),
                        create_card_col("card6", "🔥 Ocean Heat", "+337 zettajoules"),
                    ],
                    className="g-0 d-flex align-items-center",
                    style={}
                )
            ]
        ),

        #Charts    
        html.Div(
            [
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dcc.Graph(id="fig1", figure=fig)
                            ],
                            **CHART_COL_STYLE
                        ),
                        dbc.Col(
                            [
                                dcc.Graph(id="fig2")
                            ],
                            **CHART_COL_STYLE
                        )
                    ]
                ),
                dbc.Row(
                    [
                        dbc.Col(
                            [
                                dcc.Graph(id="fig3")
                            ],
                            **CHART_COL_STYLE
                        ),
                        dbc.Col(
                            [
                                dcc.Graph(id="fig4")
                            ],
                            **CHART_COL_STYLE
                        )
                    ]
                )
            ]
            ,style={}
        ),
    ],

)

# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

@app.callback(
    [
        Output('card1', 'children'),
        Output('card2', 'children'),
        Output('card3', 'children'),
        Output('card4', 'children'), 
        Output('card5', 'children'),
        Output('card6', 'children'),
        Output('fig1', 'figure'),
        Output('fig2', 'figure'), 
        Output('fig3', 'figure'),
        Output('fig4', 'figure'), 
    ],
    [
        Input('entity', 'value'),
        Input('scenario', 'value')
    ]
    
)

def multi_outputs(entity, scenario):
    if entity is None and scenario is None:
        raise PreventUpdate
        
    # Get HKPIs dataframe
    df = df_hkpis.copy()
    df = df[(df["ENTITY"] == entity) & (df["SCENARIO"].astype(str) == scenario)]
    
    # Return card data
    card1 = df.loc[df["LABEL"] == "🧬 Carbon Dioxyde", "VALUE_D"]
    card2 = df.loc[df["LABEL"] == "🌡 Temperature", "VALUE_D"]
    card3 = df.loc[df["LABEL"] == "❄️ Artic Sea Ice", "VALUE_D"]
    card4 = df.loc[df["LABEL"] == "🧊 Ice Sheet", "VALUE_D"]
    card5 = df.loc[df["LABEL"] == "🌊 Sea Level", "VALUE_D"]
    card6 = df.loc[df["LABEL"] == "🔥 Ocean Heat", "VALUE_D"]
    
    # Get graph dataframe
    df1 = df_barline.copy()
    df1 = df1[(df1["ENTITY"] == entity) & (df1["SCENARIO"].astype(str) == scenario)]
    
    # Create graph
    fig1 = create_barlinechart(df1, yaxis_title_r="Variation", yaxis_title_l="Value")
    fig2 = create_barlinechart(df1, yaxis_title_r="Variation", yaxis_title_l="Value")
    fig3 = create_barlinechart(df1, yaxis_title_r="Variation", yaxis_title_l="Value")
    fig4 = create_barlinechart(df1, yaxis_title_r="Variation", yaxis_title_l="Value")
    return card1, card2, card3, card4, card5, card6, fig1, fig2, fig3, fig4

## Output

### Generate URL and show logs

In [ ]:
if __name__ == '__main__':
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")